In [1]:
import gtfstk as gt
import pandas as pd
import geopandas as gpd
import numpy as np
import tkinter as tk
import requests, zipfile, io, os, xlsxwriter
from tkinter import filedialog

In [2]:
zip_file_url=r"http://valleyregionaltransit.org/gtfs/VRT_Transit1.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("gtfs")
gtfs_filename=os.path.basename(zip_file_url)
gtfs_filename=os.path.splitext(gtfs_filename)[0]
feed=gt.read_gtfs("gtfs",dist_units='mi')

In [3]:
route_names=feed.routes['route_short_name']
route_namesArray=route_names.unique()
route_names={i:route_namesArray[i] for i in range(len(route_names.unique()))}
route_names.update([((len(route_names)+1),'All')])
route_names=route_names.values()

In [4]:
def button_gtfs_clicked():
    root.gtfs=filedialog.askopenfilename(initialdir="/",title="Select GTFS Zip file",filetypes = (("zip files","*.zip"),("all files","*.*")))
def button_mbsl_clicked():
    root.mbsl=filedialog.askopenfilename(initialdir=r"N:\Planning - New File Structure\MASTER BUS STOP LIST",title="Select Master Bus Stop List")
def button_outfolder_clicked():
    root.outfolder=filedialog.askdirectory(initialdir="/",title="Select Output folder")
def close_window():
    root.route=tkvar.get()
    root.destroy()

root=tk.Tk()
root.title('Stops Changes')

tkvar=tk.StringVar(root)
tkvar.set("All")

label_gtfs=tk.Label(root,text="Select GTFS zip file.")
label_gtfs.grid(column=0,row=0)
button_gtfs=tk.Button(root,text="Browse",command=button_gtfs_clicked)
button_gtfs.grid(column=0,row=1)

label_mbsl=tk.Label(root,text="Select Master Bus Stop List.")
label_mbsl.grid(column=0,row=2)
button_mbsl=tk.Button(root,text="Browse",command=button_mbsl_clicked)
button_mbsl.grid(column=0,row=3)

label_out=tk.Label(root,text="Select an output folder")
label_out.grid(column=0,row=4)
button_out=tk.Button(root,text="Browse",command=button_outfolder_clicked)
button_out.grid(column=0,row=5)

route_label=tk.Label(root,text="Pick a Route")
route_label.grid(column=0,row=6)
route_choice=tk.OptionMenu(root,tkvar,*route_names)
route_choice.grid(column=0,row=7)

button_run=tk.Button(root,text='Run',width=25,command=close_window)
button_run.grid(column=0,row=8)
root.mainloop()

In [5]:
root.gtfs

'N:/Planning - New File Structure/GIS/Data/OFFICIAL CURRENT/GTFS/VRT_Transit1.zip'

In [6]:
root.mbsl

'N:/Planning - New File Structure/MASTER BUS STOP LIST/20190604 Master Bus Stop List.xlsx'

In [7]:
root.outfolder
outfolder=root.outfolder+"/{}.xlsx"

In [8]:
feed=gt.read_gtfs(root.gtfs,dist_units='mi')
MBSL=pd.read_excel(root.mbsl,sheet_name='Master Amenities List',skiprows=2,dtype={'Stop Id':str},na_values=False)
MBSL=MBSL[['Stop Id','Ada Accessible','Shelter']].fillna(0)
gtfs_filename=os.path.basename(root.gtfs)
gtfs_filename=os.path.splitext(gtfs_filename)[0]

In [9]:
def getRouteStops(route_short_name):
    df=feed.stop_times.merge(feed.trips).merge(feed.routes).merge(feed.stops)
    df=df.loc[df['route_short_name']==route_short_name]
#     df=df.groupby(['stop_id','direction_id'],as_index=False).first()
    return df

In [10]:
def getDuplicateColumns(df):
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
 
    return list(duplicateColumnNames)

In [11]:
def getUniquePatterns(route_short_name):
    route=getRouteStops(route_short_name)
    route=route.pivot(index='stop_sequence',columns='trip_id',values=['stop_id'])
    route=route.drop(getDuplicateColumns(route),axis=1)
    alpha=[]
    for letter in range(65,(65+len(route.columns))):
        alpha.append(chr(letter))
    route.columns=alpha
    route=pd.DataFrame(route.unstack()).reset_index()
    route.rename({'level_0':'Pattern',0:'stop_id'},axis=1,inplace=True)
    route.sort_values(['Pattern','stop_sequence'],inplace=True)
    return route

In [12]:
def timepointer(route_short_name):
    route=feed.stop_times.merge(feed.trips).merge(feed.routes).merge(feed.stops)
    route=route.loc[route['route_short_name']==route_short_name]
    route=route.groupby(['direction_id','shape_id','stop_sequence'],as_index=False).first()
    route=route.loc[route['departure_time'].notna()]
    route=route[['stop_name','shape_id','direction_id','stop_sequence','departure_time','stop_id']]
    route['stop_label']=route['stop_name']
    route['stop_label'].replace(r"[NSEW][NSEW][CM]","",regex=True,inplace=True)
    firsts=pd.DataFrame(route.loc[~route['stop_label'].duplicated()][['stop_label','stop_sequence','direction_id','stop_id']]).reset_index()
    firsts['number']=firsts.index+1
    dups=pd.DataFrame(route.loc[route['stop_label'].duplicated()][['stop_label','stop_sequence','direction_id','stop_id']]).reset_index()
    dups2=dups.merge(firsts,on='stop_label')
    dups2=dups2[['stop_label','number']]
    dups2[['direction_id','stop_sequence','stop_id']]=dups[['direction_id','stop_sequence','stop_id']]
    df2=firsts.append(dups2)
    df2=df2.groupby(['stop_id','direction_id'],as_index=False).first()
    df2.sort_values(['direction_id','stop_sequence'],inplace=True)
    df2=df2[['stop_id','stop_label','number']]
    return df2

In [13]:
def otherRoutes(route_short_name):
    df=feed.stop_times.merge(feed.trips)
    df=df.merge(feed.routes)
    route=df.loc[df['route_short_name']==route_short_name]
    route=route.groupby('stop_id',as_index=False).first()
    df=df.loc[df['route_short_name']!=route_short_name]
    df=df.groupby(['stop_id','route_short_name'],as_index=False).first()
    df=pd.DataFrame(df.groupby('stop_id')['route_short_name'].agg(lambda x: ', '.join(x))).reset_index()
    df=df[['stop_id','route_short_name']]
    df=df.rename({'route_short_name':'other_routes'},axis=1)
    route=route.merge(df,how='left')
    route=route[['stop_id','other_routes']]
    return route

In [14]:
def MergeMBSL(route_short_name):
    df=getRouteStops(route_short_name)
    df=df.merge(MBSL,left_on='stop_id',right_on='Stop Id')
    df=df[['stop_id','Ada Accessible','Shelter']]
    return df

In [15]:
def stopLister(route_short_name, save_folder=None):
    df=getUniquePatterns(route_short_name)
    df=df.merge(feed.stops,how='left').merge(timepointer(route_short_name),how='left').merge(otherRoutes(route_short_name),how='left').merge(MergeMBSL(route_short_name),how='left')
    df=df.groupby(['Pattern','stop_sequence','stop_id'],as_index=False).first()
    df=df[['Pattern','stop_sequence','stop_id','number','stop_name','Ada Accessible','Shelter','other_routes']]
    df.rename({'number':'Timepoint','stop_name':'Stop Name','Ada Accessible':'Accessible','other_routes':'Other Routes'},axis=1,inplace=True)
    if save_folder:
        path=save_folder
        writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
        for i in df.Pattern.unique():
            out=df.loc[df['Pattern']==i]
            out.to_excel(writer,sheet_name=i,index=False)
        writer.save()
        writer.close()

In [16]:
if root.route=='All':
    for i in feed.routes.route_short_name.unique():
        stopLister(i,outfolder.format(i))
else:
    stopLister(root.route,outfolder.format(root.route))

c:\users\bparker\appdata\local\conda\conda\envs\py3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
